# Merge HRV Metrics

Purpose of this notebook is to load all the calculate HRV metrics from the Excel sheets and to concatenate them into a single Excel sheet. This facilitates data analysis.

In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np
from typing import Union, List

## Parameters

In [ ]:
WORKING_DIR = Path().cwd()
ROOT_DIR = WORKING_DIR.parent
DATA_DIR = ROOT_DIR / 'data'
PROCESSED_DATA_DIR = DATA_DIR / 'processed'
HRV_DATA_DIR = DATA_DIR / 'hrv' # the output directory for the concatenated hrv data
HRV_DATA_DIR.mkdir(exist_ok=True, parents=True)
REPORTS_DIR = ROOT_DIR / 'reports'

## Support functions

In [ ]:
def plausible_to_nan(
    df: pd.DataFrame, 
    column: str, 
    lower_bound: float = 9, 
    upper_bound: float = 110
) -> pd.DataFrame:
    """
    Set implausible values in a specified DataFrame column to NaN based on user-defined bounds.
    
    Parameters:
    ----------
    df : pd.DataFrame
        The input DataFrame containing the data to process.
    column : str
        The name of the column to check for plausibility.
    lower_bound : float, optional
        The lower bound for plausible values. Values below this will be set to NaN. Default is 9.
    upper_bound : float, optional
        The upper bound for plausible values. Values above this will be set to NaN. Default is 110.
        
    Returns:
    -------
    pd.DataFrame
        A DataFrame with implausible values in the specified column replaced by NaN.
    """
    df = df.copy()  # To avoid modifying the original DataFrame
    df[column] = np.where(
        (df[column] < lower_bound) | (df[column] > upper_bound), 
        np.nan, 
        df[column]
    )
    return df

def identify_outliers_zscore(
    data: Union[List[float], np.ndarray, pd.Series], 
    threshold: float = 1.96
) -> List[bool]:
    """
    Identify outliers in a dataset based on the Z-score method.
    
    Parameters:
    ----------
    data : Union[List[float], np.ndarray, pd.Series]
        The input data, which can be a list, numpy array, or pandas series.
    threshold : float, optional
        The Z-score threshold to determine outliers. Default is 1.96 (i.e., >95%).
        
    Returns:
    -------
    List[bool]
        A list of booleans indicating whether each value is an outlier (True) or not (False).
        NaN values are treated as outliers.
    """
    # Convert data to a numpy array
    data_array = np.asarray(data, dtype=np.float64)
    
    # Identify NaN values (NaN will be treated as outliers)
    nan_mask = np.isnan(data_array)
    
    # Calculate Z-scores only for non-NaN values
    valid_data = data_array[~nan_mask]
    mean = valid_data.mean()
    std = valid_data.std(ddof=0)
    
    z_scores = (valid_data - mean) / std
    outliers_non_nan = np.abs(z_scores) > threshold
    
    # Create a boolean array for the entire data, marking NaNs as outliers
    outliers = np.full(data_array.shape, True)  # Default to True for NaN values
    outliers[~nan_mask] = outliers_non_nan
    
    return outliers.tolist()

def replace_outliers_zscore(
    data: Union[List[float], np.ndarray, pd.Series], 
    outliers: List[bool], 
    method: str = "median"
) -> Union[np.ndarray, pd.Series]:
    """
    Replace outliers in a dataset based on Z-score detection with the mean or median of non-outlier values.
    
    Parameters:
    ----------
    data : Union[List[float], np.ndarray, pd.Series]
        The input data containing original values.
    outliers : List[bool]
        A list of booleans indicating whether each value is an outlier (True) or not (False).
    method : str, optional
        The method to replace outliers, either 'median' or 'mean'. Default is 'median'.
        
    Returns:
    -------
    Union[np.ndarray, pd.Series]
        The dataset with outliers replaced by the specified method's value.
    """
    # Convert data to a numpy array for consistent processing
    data_array = np.asarray(data)
    
    # Validate the method parameter
    if method not in {"median", "mean"}:
        raise ValueError("Invalid method. Use 'median' or 'mean'.")
    
    # Compute the replacement value based on the specified method
    non_outliers = data_array[~np.array(outliers)]
    replacement_value = np.median(non_outliers) if method == "median" else np.mean(non_outliers)
    
    # Replace outliers with the calculated value
    data_array[outliers] = replacement_value
    
    # Return the modified data in the same type as the input
    if isinstance(data, pd.Series):
        return pd.Series(data_array, index=data.index)
    elif isinstance(data, list):
        return data_array.tolist()
    return data_array

def identify_clean_outliers(
    df: pd.DataFrame, 
    hrv_variable_name: str = "HRV_RMSSD", 
    method: str = "median", 
    threshold_z_score: float = 1.96
) -> pd.DataFrame:
    """
    Identifies outliers in a specified HRV variable using the IQR method and replaces them with 
    the median or mean of the non-outlier values.
    
    Parameters:
    ----------
    df : pd.DataFrame
        The input DataFrame containing the data to process. 
        Must include 'segment_name' and the specified `hrv_variable_name`.
    hrv_variable_name : str, optional
        The name of the HRV variable to process. Default is "HRV_RMSSD".
    method : str, optional
        The method to replace outliers, either "median" or "mean". Default is "median".
    threshold_z_score : float, optional
        The Z-score threshold to determine outliers. Default is 1.96 (i.e., >95%).
        
    Returns:
    -------
    pd.DataFrame
        The original DataFrame with two additional columns:
        - `{hrv_variable_name}_outlier`: Boolean column indicating outliers.
        - `{hrv_variable_name}_cleaned`: Column with outliers replaced by the specified method.
    """
    # Ensure method is valid
    if method not in {"median", "mean"}:
        raise ValueError("Invalid method. Use 'median' or 'mean'.")

    for segment in df['segment_name'].unique():
        subset_df = df[df['segment_name'] == segment].copy()

        rmssd_values = subset_df[hrv_variable_name]
        
        # Identify outliers
        subset_df[f'{hrv_variable_name}_outlier'] = identify_outliers_zscore(rmssd_values, threshold=threshold_z_score)
        
        # Replace outliers
        subset_df[f'{hrv_variable_name}_cleaned'] = replace_outliers_zscore(
            rmssd_values, 
            subset_df[f'{hrv_variable_name}_outlier'], 
            method=method
        )

        # Update the original DataFrame
        df.loc[df['segment_name'] == segment, f'{hrv_variable_name}_outlier'] = subset_df[f'{hrv_variable_name}_outlier'].values
        df.loc[df['segment_name'] == segment, f'{hrv_variable_name}_cleaned'] = subset_df[f'{hrv_variable_name}_cleaned'].values

    return df





In [ ]:
def detect_segment_level_outliers(
    df: pd.DataFrame, 
    hrv_variable_name: str = "HRV_RMSSD", 
    cv_threshold: float = 0.75
) -> pd.DataFrame:
    """
    Detects segment-level outliers by checking the coefficient of variation (CV) for each segment.
    Also flags segments as outliers if the standard deviation is zero (indicating no variability).
    
    Parameters:
    ----------
    df : pd.DataFrame
        The input DataFrame containing the data to process.
    hrv_variable_name : str
        The name of the HRV variable to process.
    cv_threshold : float, optional
        Threshold for the coefficient of variation. Segments with CV greater than this will be flagged.
        
    Returns:
    -------
    pd.DataFrame
        Original DataFrame with an additional column 'segment_outlier' indicating segment-level outliers.
    """
    segment_outliers = []
    
    for segment in df['segment_name'].unique():
        segment_data = df[df['segment_name'] == segment][hrv_variable_name]
        
        # Calculate mean and standard deviation
        mean = segment_data.mean()
        std = segment_data.std()
        
        # Calculate Coefficient of Variation
        cv = std / mean if mean != 0 else np.inf
        
        # Determine if the segment is an outlier
        is_outlier = (cv > cv_threshold) or (std == 0)
        segment_outliers.extend([is_outlier] * len(segment_data))
    
    df['segment_outlier'] = segment_outliers
    return df


## Identify data

In [ ]:
# Get all non-empty directories in the processed data directory since they contain (hopefully) the HRV metrics
all_items = PROCESSED_DATA_DIR.glob("*/")
all_dirs = [x for x in all_items if x.is_dir()]
non_empty_dirs = [dir for dir in all_dirs if any(dir.iterdir())]

## Concatenate all HRV Excel files

In [42]:
# In each directory, look for xlsx files, read them, and concatenate them into a single df
n_files = 0 # to keep track of the number of excel files we read and process
hrv_df = pd.DataFrame()
for directory in non_empty_dirs:
    xlsx_files = [file for file in directory.glob('*.xlsx') if not file.name.startswith('~$')]
    n_files += len(xlsx_files)
    for file in xlsx_files:
        df = pd.read_excel(file)
        df_cleaned = plausible_to_nan(df, column = "HRV_RMSSD" , lower_bound=9, upper_bound=130)
        df_cleaned = identify_clean_outliers(df_cleaned, threshold_z_score=1.96, hrv_variable_name = 'HRV_RMSSD_plausible', method = "mean")
        df_cleaned = identify_clean_outliers(df, threshold_z_score=1.96, hrv_variable_name = 'HRV_RMSSD_cleaned', method = "mean") # second round
        df_cleaned = detect_segment_level_outliers(df_cleaned, cv_threshold=0.7, hrv_variable_name='HRV_RMSSD_cleaned_cleaned')
        hrv_df = pd.concat([hrv_df, df_cleaned])

# Export
hrv_df.to_excel(HRV_DATA_DIR / 'cleaned_hrv_data.xlsx', index=False)

KeyError: 'HRV_RMSSD_plausible'

Semgnet-level detection should return outlier if all values are the same as well (e.g., NaNs)

In [ ]:
plausible_df = plausible_to_nan(df, column = "HRV_RMSSD" , lower_bound=9, upper_bound=30)

outliers = identify_outliers_zscore(plausible_df['HRV_RMSSD'], threshold=1.96)
cleaned = replace_outliers_zscore(plausible_df['HRV_RMSSD'], outliers, method='mean')

In [ ]:
df.shape

In [ ]:
identify_clean_outliers(plausible_df, method="mean")